In [39]:
# imports
import requests
import urllib
import pandas as pd
import json
import time
import random
import base64

In [40]:
# save variables
url = "https://api.spotify.com/v1/"
authToken = 'BQBtdH8ckkL3n7echpYPbRLhQXDi0TxKN65dxSWxShxXNCxCXxCViuSPWa3Rsr1I91gFZLS_DzE0aHlo5KGf2XD1OL05nJNfAwskljtdZSxIqao-adkaaUTdTG1Nk4A6MlvcAMjNXH6ccs6MvxKP3NAwfLsEU2J-ON0ZyicofOhaNtAeV6IQqA'
getHeader = {
    'Accept': 'application/json',
    'Content-Type': 'application/json',
    'Authorization': 'Bearer ' + authToken
}

# Repull top 100 songs for 2020 and 2021
Using:
https://open.spotify.com/playlist/1GIJsNcI4aLIm0ftqRFZVu?si=a7d9a79f75d14ac8 (2020)
https://open.spotify.com/playlist/1bjAgktyYGqKJXynU4H8SO?si=94095498f3e6465b (2021)

In [45]:
playlists = [{'year': 2020, 'id': '1GIJsNcI4aLIm0ftqRFZVu'}, {'year': 2021, 'id': '1bjAgktyYGqKJXynU4H8SO'}]

newPlaylistTracks = []

# get all of the tracks
for playlist in playlists:
    resp = requests.get('https://api.spotify.com/v1/playlists/' + playlist['id'] + '/tracks?limit=100', headers=getHeader)
    rawTracks = json.loads(resp.content.decode('utf-8'))["items"]

    for rawTrack in rawTracks:
        trackObject = {}
        trackObject['Title'] = rawTrack['track']['name']
        trackObject['trackId'] = rawTrack['track']['id']
        trackObject['Popularity'] = rawTrack['track']['popularity']
        trackObject['Artist'] = rawTrack['track']['artists'][0]['name']
        trackObject['artistId'] = rawTrack['track']['artists'][0]['id']
        trackObject['Top Year'] = playlist['year']
        trackObject['Release Date'] = rawTrack['track']['album']['release_date'][0:4]
        
        newPlaylistTracks.append(trackObject)

# get genres and musical features       
for track in newPlaylistTracks:
    
    # get genres
    genreResp = requests.get('https://api.spotify.com/v1/artists/' + track['artistId'], headers=getHeader)
    
    if len(json.loads(genreResp.content.decode('utf-8'))["genres"]) > 0:
        genre = json.loads(genreResp.content.decode('utf-8'))["genres"][0]
    else:
        genre = "n/a"
        
    track['Genre'] = genre
    
    # get musical features
    featureResp = requests.get('https://api.spotify.com/v1/audio-features/' + track['trackId'], headers=getHeader)
    features = json.loads(featureResp.content.decode('utf-8'))
    track['Speechiness'] = features['speechiness']
    track['Acousticness'] = features['acousticness']
    track['Liveness'] = features['liveness']
    track['Danceability'] = features['danceability']
    track['Energy'] = features['energy']
    track['Loudness'] = features['loudness']
    track['Valence'] = features['valence']
    track['Duration'] = features['duration_ms']
    track['BPM'] = features['tempo']
    
print(newPlaylistTracks)

[{'Title': 'Blinding Lights', 'trackId': '0VjIjW4GlUZAMYd2vXMi3b', 'Popularity': 90, 'Artist': 'The Weeknd', 'artistId': '1Xyo4u8uXC1ZmMpatF05PJ', 'Top Year': 2020, 'Release Date': '2020', 'Genre': 'canadian contemporary r&b', 'Speechiness': 0.0598, 'Acousticness': 0.00146, 'Liveness': 0.0897, 'Danceability': 0.514, 'Energy': 0.73, 'Loudness': -5.934, 'Valence': 0.334, 'Duration': 200040, 'BPM': 171.005}, {'Title': 'Circles', 'trackId': '21jGcNKet2qwijlDFuPiPb', 'Popularity': 85, 'Artist': 'Post Malone', 'artistId': '246dkjvS1zLTtiykXe5h60', 'Top Year': 2020, 'Release Date': '2019', 'Genre': 'dfw rap', 'Speechiness': 0.0395, 'Acousticness': 0.192, 'Liveness': 0.0863, 'Danceability': 0.695, 'Energy': 0.762, 'Loudness': -3.497, 'Valence': 0.553, 'Duration': 215280, 'BPM': 120.042}, {'Title': 'The Box', 'trackId': '0nbXyq5TXYPCO7pr3N8S4I', 'Popularity': 80, 'Artist': 'Roddy Ricch', 'artistId': '757aE44tKEUQEqRuT6GnEB', 'Top Year': 2020, 'Release Date': '2019', 'Genre': 'melodic rap', 'Spe

In [54]:
# Merge with original data set
# load original data
df = pd.read_csv("data.csv")
df = df[['Title', 'Artist', 'Genre', 'Release Date', 'Popularity', 'Danceability', 'Speechiness', 'Energy', 'Loudness (dB)', 'Acousticness', 'Liveness', 'BPM', 'Duration (ms)', 'Valence', 'Top Year']]
df = df.rename(columns={"Loudness (dB)": "Loudness", "Duration (ms)": "Duration"})

# merge with new data
df = df.append(newPlaylistTracks)    

print(df.head())

# export to csv
df.to_csv("all_data.csv")

                                           Title      Artist        Genre  \
0                  STARSTRUKK (feat. Katy Perry)       3OH!3    dance pop   
1                    My First Kiss (feat. Ke$ha)       3OH!3    dance pop   
2                                I Need A Dollar  Aloe Blacc     pop soul   
3  Airplanes (feat. Hayley Williams of Paramore)       B.o.B  atl hip hop   
4              Nothin' on You (feat. Bruno Mars)       B.o.B  atl hip hop   

  Release Date  Popularity  Danceability  Speechiness  Energy  Loudness  \
0         2009        70.0          0.61         0.06    0.81      -6.0   
1         2010        68.0          0.68         0.08    0.89      -4.0   
2         2010        72.0          0.84         0.03    0.48      -7.0   
3         2010        80.0          0.66         0.12    0.87      -4.0   
4         2010        79.0          0.69         0.05    0.85      -6.0   

   Acousticness  Liveness    BPM  Duration  Valence  Top Year trackId artistId  
0    

# Get average for each characteristic every year

In [57]:
# load in data
df = pd.read_csv("all_data.csv")
df = df[['Popularity', 'Danceability', 'Speechiness', 'Energy', 'Loudness', 'Acousticness', 'Liveness', 'BPM', 'Duration', 'Valence', 'Top Year']]

# collapse for averages
df_averages = df.groupby("Top Year").agg('mean')

print(df_averages.head())



          Popularity  Danceability  Speechiness  Energy  Loudness  \
Top Year                                                            
2010           70.18        0.6529       0.0995  0.7665     -4.95   
2011           73.05        0.6356       0.0852  0.7610     -5.16   
2012           72.85        0.6401       0.0927  0.7407     -5.53   
2013           73.81        0.6355       0.0816  0.7244     -5.63   
2014           74.32        0.6588       0.0765  0.7103     -5.46   

          Acousticness  Liveness     BPM  Duration  Valence  
Top Year                                                     
2010            0.0962     20.66  119.56  226450.0   0.5674  
2011            0.1174     18.76  127.29  230190.0   0.5485  
2012            0.0970     19.71  123.10  226380.0   0.5329  
2013            0.1152     19.13  119.91  229490.0   0.5305  
2014            0.1229     18.66  119.73  227900.0   0.5095  


# Get #1 song every year

In [36]:
df_topSong = pd.read_csv("top_song.csv")

# convert from df to array
topSongs = df_topSong.to_dict('records')

# define url
uri = url + "search"
uri2 = url + "artists/"
pairedTopSongs = []
failedTopSongs = []

# loop through songs
for song in topSongs:
    title = song['Title']
    artist = song['Artist']
    
    params = {
        'query': title,
        'type': 'track',
        'limit': 1,
        'offset': 0
    }
    
    response = requests.get(uri, headers=getHeader, params=params)

    
    if len(json.loads(response.content)['tracks']['items']) > 0:
        print(title, artist, json.loads(response.content)['tracks']['items'][0]['id'])
        songId = json.loads(response.content)['tracks']['items'][0]['id']
        artistId = json.loads(response.content)['tracks']['items'][0]['album']['artists'][0]['id']
        
        # get artist now
        response2 = requests.get(uri2 + artistId, headers=getHeader)
        
        if len(json.loads(response2.content.decode('utf-8'))["genres"]) > 0:
            genre = json.loads(response2.content.decode('utf-8'))["genres"][0]
        else:
            genre = "n/a"
        
        # append new object
        songObject = {
            'title': title, 
            'songId': songId, 
            'artistId': artistId,
            'genre': genre
        }
        print(songObject)
        
        pairedTopSongs.append(songObject)
        
    else:
        failedTopSongs.append({'title': title, 'artist': artist})
        
print(pairedTopSongs)

drivers license Olivia Rodrigo 5wANPM4fQCJwkGd4rN57mH
{'external_urls': {'spotify': 'https://open.spotify.com/artist/1McMsnEElThX1knmY4oliG'}, 'followers': {'href': None, 'total': 21798180}, 'genres': ['pop'], 'href': 'https://api.spotify.com/v1/artists/1McMsnEElThX1knmY4oliG', 'id': '1McMsnEElThX1knmY4oliG', 'images': [{'height': 640, 'url': 'https://i.scdn.co/image/ab6761610000e5ebee954a3b5418065c2fe253fb', 'width': 640}, {'height': 320, 'url': 'https://i.scdn.co/image/ab67616100005174ee954a3b5418065c2fe253fb', 'width': 320}, {'height': 160, 'url': 'https://i.scdn.co/image/ab6761610000f178ee954a3b5418065c2fe253fb', 'width': 160}], 'name': 'Olivia Rodrigo', 'popularity': 83, 'type': 'artist', 'uri': 'spotify:artist:1McMsnEElThX1knmY4oliG'}
{'title': 'drivers license', 'songId': '5wANPM4fQCJwkGd4rN57mH', 'artistId': '1McMsnEElThX1knmY4oliG', 'genre': 'pop'}
Blinding Lights The Weeknd 0VjIjW4GlUZAMYd2vXMi3b
{'external_urls': {'spotify': 'https://open.spotify.com/artist/1Xyo4u8uXC1ZmMpat

Pumped Up Kicks Foster The People 7w87IxuO7BDcJ3YUqCyMTT
{'external_urls': {'spotify': 'https://open.spotify.com/artist/7gP3bB2nilZXLfPHJhMdvc'}, 'followers': {'href': None, 'total': 3550057}, 'genres': ['indietronica', 'modern rock', 'rock'], 'href': 'https://api.spotify.com/v1/artists/7gP3bB2nilZXLfPHJhMdvc', 'id': '7gP3bB2nilZXLfPHJhMdvc', 'images': [{'height': 640, 'url': 'https://i.scdn.co/image/ab6761610000e5ebb5a3c9a9ca40824e0f44ea78', 'width': 640}, {'height': 320, 'url': 'https://i.scdn.co/image/ab67616100005174b5a3c9a9ca40824e0f44ea78', 'width': 320}, {'height': 160, 'url': 'https://i.scdn.co/image/ab6761610000f178b5a3c9a9ca40824e0f44ea78', 'width': 160}], 'name': 'Foster The People', 'popularity': 70, 'type': 'artist', 'uri': 'spotify:artist:7gP3bB2nilZXLfPHJhMdvc'}
{'title': 'Pumped Up Kicks', 'songId': '7w87IxuO7BDcJ3YUqCyMTT', 'artistId': '7gP3bB2nilZXLfPHJhMdvc', 'genre': 'indietronica'}
TiK ToK Ke$ha 0HPD5WQqrq7wPWR7P7Dw1i
{'external_urls': {'spotify': 'https://open.spo

# Pull missing genres

In [48]:
# load data
df = pd.read_csv("data.csv")

# keep only title, artist, genre columns
df_blankSongs = df[['Title', 'Artist', 'Genre']]

# keep only records with blank genres (we'll be filling them in)
# there are no blank songs after dropping 2020 and 2021
df_blankSongs = df_blankSongs.loc[df_blankSongs['Genre'].isnull()]

# convert from df to array
blankSongs = df_blankSongs.to_dict('records')
print(blankSongs)

                                             Title              Artist  \
0                    STARSTRUKK (feat. Katy Perry)               3OH!3   
1                      My First Kiss (feat. Ke$ha)               3OH!3   
2                                  I Need A Dollar          Aloe Blacc   
3    Airplanes (feat. Hayley Williams of Paramore)               B.o.B   
4                Nothin' on You (feat. Bruno Mars)               B.o.B   
..                                             ...                 ...   
995                                     SICKO MODE        Travis Scott   
996                                      EARFQUAKE  Tyler, The Creator   
997                      Boasty (feat. Idris Elba)               Wiley   
998                    Strike a Pose (feat. Aitch)    Young T & Bugsey   
999      The London (feat. J. Cole & Travis Scott)          Young Thug   

           Genre Release Date  Popularity  Danceability  dnce(raw ignore)  \
0      dance pop         2009     

# Get Track IDs & genres for missing (2010-2019 only)
URL: https://api.spotify.com/v1/search

In [156]:
uri = url + "search"
uri2 = url + "artists/"
pairedSongs = []
failedSongs = []

for song in blankSongs:
    title = song['Title']
    artist = song['Artist']
    
    params = {
        'query': title,
        'type': 'track',
        'limit': 1,
        'offset': 0
    }
    
    response = requests.get(uri, headers=getHeader, params=params)

    
    if len(json.loads(response.content)['tracks']['items']) > 0:
        print(title, artist, json.loads(response.content)['tracks']['items'][0]['id'])
        songId = json.loads(response.content)['tracks']['items'][0]['id']
        artistId = json.loads(response.content)['tracks']['items'][0]['album']['artists'][0]['id']
        
        # get artist now
        response2 = requests.get(uri2 + artistId, headers=getHeader)
        
        if len(json.loads(response2.content.decode('utf-8'))["genres"]) > 0:
            genre = json.loads(response2.content.decode('utf-8'))["genres"][0]
        else:
            genre = "n/a"
        
        # append new object
        songObject = {
            'title': title, 
            'songId': songId, 
            'artistId': artistId,
            'genre': genre
        }
        print(songObject)
        
        pairedSongs.append(songObject)
        
    else:
        failedSongs.append({'title': title, 'artist': artist})

NOT SOBER (feat. Polo G & Stunna Gambino) The Kid LAROI 1JXBNvlDEl2epKSJ9EOgKX
{'title': 'NOT SOBER (feat. Polo G & Stunna Gambino)', 'songId': '1JXBNvlDEl2epKSJ9EOgKX', 'artistId': '2tIP7SsRs7vjIcLrU85W8J', 'genre': 'australian hip hop'}
Build a Bitch Bella Poarch 7BoobGhD4x5K96Me0hqC8Q
{'title': 'Build a Bitch', 'songId': '7BoobGhD4x5K96Me0hqC8Q', 'artistId': '26cMerAxjx9GedFt0lMDjm', 'genre': 'modern alternative pop'}
Tiroteo - Remix Marc Seguí, Rauw Alejandro, Pol Granch 4OwhwvKESFtuu06dTgct7i
{'title': 'Tiroteo - Remix', 'songId': '4OwhwvKESFtuu06dTgct7i', 'artistId': '5FQ8tBUtIamA2hRtatrYUF', 'genre': 'spanish pop'}
Meu Pedaço de Pecado João Gomes 2kJRlOyrxsy7Ot3c5TZ8Q6
{'title': 'Meu Pedaço de Pecado', 'songId': '2kJRlOyrxsy7Ot3c5TZ8Q6', 'artistId': '4JNo6Q5KdcRf1vtSX9mB0S', 'genre': 'arrocha'}
Don't Start Now Dua Lipa 3PfIrDoz19wz7qK7tYeu62
{'title': "Don't Start Now", 'songId': '3PfIrDoz19wz7qK7tYeu62', 'artistId': '6M2wZ9GZgrQXHCFfjv46we', 'genre': 'dance pop'}
Dinero Trinida

MODO TURBO Luísa Sonza, Pabllo Vittar, Anitta 2ip4hxYupn3CSsHjn4l2a8
{'title': 'MODO TURBO', 'songId': '2ip4hxYupn3CSsHjn4l2a8', 'artistId': '4PzYKhC14sTJNEr0dzoo0d', 'genre': 'funk carioca'}
Amor ou o Litrão Petter Ferraz, Menor Nico 7qmnP8Woc47dLSrMD5MxL1
{'title': 'Amor ou o Litrão', 'songId': '7qmnP8Woc47dLSrMD5MxL1', 'artistId': '6gGJH7joTDx9M3xREiAq9U', 'genre': 'n/a'}
Jingle Bell Rock Bobby Helms 7vQbuQcyTflfCIOu3Uzzya
{'title': 'Jingle Bell Rock', 'songId': '7vQbuQcyTflfCIOu3Uzzya', 'artistId': '38EmEgXkgK51MT2tPY0EoC', 'genre': 'n/a'}
Sleigh Ride The Ronettes 5ASM6Qjiav2xPe7gRkQMsQ
{'title': 'Sleigh Ride', 'songId': '5ASM6Qjiav2xPe7gRkQMsQ', 'artistId': '0LyfQWJT6nXafLPZqxe9Of', 'genre': 'n/a'}
Driving Home for Christmas - 2019 Remaster Chris Rea 2GoiggvbDKJtE2rWrdkfoC
{'title': 'Driving Home for Christmas - 2019 Remaster', 'songId': '2GoiggvbDKJtE2rWrdkfoC', 'artistId': '0LyfQWJT6nXafLPZqxe9Of', 'genre': 'n/a'}
Run Rudolph Run - Single Version Chuck Berry 6qgF7oiz11BBipygpNZT

In [160]:
# check successfully paired songs
print(len(pairedSongs))

# check failed songs
print(len(failedSongs)) # do this one manually

# save array as df
df_partial_fix = pd.DataFrame(pairedSongs, columns=['title', 'songId', 'artistId', 'genre'])
df_partial_fix.head()

85
1


,title,songId,artistId,genre
0,NOT SOBER (feat. Polo G & Stunna Gambino),1JXBNvlDEl2epKSJ9EOgKX,2tIP7SsRs7vjIcLrU85W8J,australian hip hop
1,Build a Bitch,7BoobGhD4x5K96Me0hqC8Q,26cMerAxjx9GedFt0lMDjm,modern alternative pop
2,Tiroteo - Remix,4OwhwvKESFtuu06dTgct7i,5FQ8tBUtIamA2hRtatrYUF,spanish pop
3,Meu Pedaço de Pecado,2kJRlOyrxsy7Ot3c5TZ8Q6,4JNo6Q5KdcRf1vtSX9mB0S,arrocha
4,Don't Start Now,3PfIrDoz19wz7qK7tYeu62,6M2wZ9GZgrQXHCFfjv46we,dance pop


## Match new data to old data

In [166]:
# all data in df
# fixed data in df_partial_fix

title_genre_dict = {}

for row in pairedSongs:
        title_genre_dict[row['title']] = row['genre']
            
print(title_genre_dict)
        
missing_mask = df['Genre'].isna()
df.loc[missing_mask, 'Genre'] = df.loc[missing_mask, 'Title'].map(title_genre_dict)

df.head()

{'NOT SOBER (feat. Polo G & Stunna Gambino)': 'australian hip hop', 'Build a Bitch': 'modern alternative pop', 'Tiroteo - Remix': 'spanish pop', 'Meu Pedaço de Pecado': 'arrocha', "Don't Start Now": 'dance pop', 'Dinero': 'n/a', 'Love Again': 'n/a', 'In Da Getto': 'reggaeton', 'No Me Conocen (Remix) [con DUKI, Rei & Tiago PZK]': 'concurso de talentos argentino', 'París': 'pop', 'Sunflower - Spider-Man: Into the Spider-Verse': 'n/a', 'Hold On': 'acoustic pop', 'Aquelas Coisas': 'arrocha', 'Não, Não Vou': 'arrocha', 'Como Si No Importara': 'pop argentino', 'Se For Amor': 'arrocha', 'Ramenez la coupe à la maison': 'francoton', '34+35': 'dance pop', 'bad guy': 'art pop', 'As the World Caves In': 'gen z singer-songwriter', 'Chosen (feat. Ty Dolla $ign)': 'pop rap', 'Lost Cause': 'art pop', 'Dick (feat. Doja Cat)': 'n/a', 'malibu': 'dance pop', 'Wellerman - Sea Shanty / 220 KID x Billen Ted Remix': 'n/a', 'Anyone': 'canadian pop', 'Therefore I Am': 'art pop', 'Higher Power': 'n/a', 'BICHOTA'

,Title,Artist,Genre,Release Date,Popularity,Danceability,dnce(raw ignore),Energy,nrgy(raw ignore),Loudness (dB),...,Acousticness,acous(ignore),Liveness,BPM,Duration (ms),dur (ignore),Valence,val(ignore),Top Year (ignore),Top Year
0,STARSTRUKK (feat. Katy Perry),3OH!3,dance pop,2009,70.0,0.61,61.0,0.81,81.0,-6.0,...,0.00,0.0,23.0,140.0,203000.0,203.0,0.23,23.0,2010,2010
1,My First Kiss (feat. Ke$ha),3OH!3,dance pop,2010,68.0,0.68,68.0,0.89,89.0,-4.0,...,0.01,1.0,36.0,138.0,192000.0,192.0,0.83,83.0,2010,2010
2,I Need A Dollar,Aloe Blacc,pop soul,2010,72.0,0.84,84.0,0.48,48.0,-7.0,...,0.20,20.0,9.0,95.0,243000.0,243.0,0.96,96.0,2010,2010
3,Airplanes (feat. Hayley Williams of Paramore),B.o.B,atl hip hop,2010,80.0,0.66,66.0,0.87,87.0,-4.0,...,0.11,11.0,4.0,93.0,180000.0,180.0,0.38,38.0,2010,2010
4,Nothin' on You (feat. Bruno Mars),B.o.B,atl hip hop,2010,79.0,0.69,69.0,0.85,85.0,-6.0,...,0.39,39.0,9.0,104.0,268000.0,268.0,0.74,74.0,2010,2010


In [170]:
df.to_csv("fixed_data.csv")